# Análisis de adjudicación de proyectos de I+D por Universidades en Chile

**Autor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />
**Github:** <https://github.com/rpmunoz> <br />


In [3]:
import numpy as np
import pandas as pd

from fuzzywuzzy import fuzz
import re, unicodedata, datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('max_colwidth', 200)

In [4]:
mineduc_file='data/Mineduc Listado IES Vigentes 05-2016.csv'
mineduc=pd.read_csv(mineduc_file, header=0, sep=',')
mineduc['fecha_reconocimiento']=pd.to_datetime(mineduc['fecha_reconocimiento'])
mineduc['año_reconocimiento']=mineduc['fecha_reconocimiento'].dt.year

In [7]:
mineduc.head(100)

,tipo,n_registro,nombre,tipo_reconocimiento,documento_reconocimiento,n_documento_reconocimiento,fecha_reconocimiento,regimen_funcionamiento,año_reconocimiento
0,C,1,Universidad Gabriela Mistral,Autorización de Funcionamiento,Certificado,S/N,1981-01-12,Autónoma,1981
1,C,2,Universidad Finis Terrae,Autorización de Funcionamiento,Certificado,S/N,1982-02-01,Autónoma,1982
2,C,3,Universidad Diego Portales,Autorización de Funcionamiento,Certificado,1082,1982-12-21,Autónoma,1982
3,C,4,Universidad Central de Chile,Autorización de Funcionamiento,Certificado,125,1983-02-24,Autónoma,1983
4,C,10,Universidad Bolivariana,Autorización de Funcionamiento,Certificado,584,1988-06-21,Autónoma,1988
5,C,12,Universidad Pedro de Valdivia,Autorización de Funcionamiento,Certificado,770,1988-08-25,Autónoma,1988
6,C,13,Universidad Mayor,Autorización de Funcionamiento,Oficio,390,1988-05-13,Autónoma,1988
7,C,14,Universidad Academia de Humanismo Cristiano,Autorización de Funcionamiento,Certificado,783,1988-08-26,Autónoma,1988
8,C,16,Universidad Santo Tomás,Autorización de Funcionamiento,Certificado,1180,1988-12-28,Autónoma,1988
9,C,19,Universidad La República,Autorización de Funcionamiento,Oficio,98,1989-02-13,Autónoma,1989


In [6]:
mineduc['tipo'].unique()

array(['C', 'A', 'B', 'D', 'E'], dtype=object)

In [9]:
for i in range(len(mineduc.index)):
    old=mineduc.loc[i,'nombre']
    new=re.sub(" ?\(\+\+\) ?| ?\(\*.*\) ?","", old)
    print("OLD: ", old)
    print("NEW: ", new)
    mineduc.loc[i,'nombre']=new

OLD:  Universidad Gabriela Mistral
NEW:  Universidad Gabriela Mistral
OLD:  Universidad Finis Terrae
NEW:  Universidad Finis Terrae
OLD:  Universidad Diego Portales
NEW:  Universidad Diego Portales
OLD:  Universidad Central de Chile
NEW:  Universidad Central de Chile
OLD:  Universidad Bolivariana
NEW:  Universidad Bolivariana
OLD:  Universidad Pedro de Valdivia
NEW:  Universidad Pedro de Valdivia
OLD:  Universidad Mayor
NEW:  Universidad Mayor
OLD:  Universidad Academia de Humanismo Cristiano
NEW:  Universidad Academia de Humanismo Cristiano
OLD:  Universidad Santo Tomás
NEW:  Universidad Santo Tomás
OLD:  Universidad La República
NEW:  Universidad La República
OLD:  Universidad Sek
NEW:  Universidad Sek
OLD:  Universidad de Las Américas
NEW:  Universidad de Las Américas
OLD:  Universidad Andrés Bello
NEW:  Universidad Andrés Bello
OLD:  Universidad de Viña del Mar
NEW:  Universidad de Viña del Mar
OLD:  Universidad Adolfo Ibañez
NEW:  Universidad Adolfo Ibañez
OLD:  Universidad Iberoa

Usaremos solamente las Universidades tipo A, B y C

In [15]:
#mineduc['tipo']=='A' | mineduc['tipo']=='B' | mineduc['tipo']=='C'

#mineduc_univ=mineduc.query("(tipo=='A' | tipo=='B' | tipo=='C') & (nombre=='Centro de Formación Técnica FINNING')")
mineduc_univ=mineduc.query("tipo=='A' | tipo=='B' | tipo=='C'")

mineduc_univ=mineduc_univ.loc[mineduc_univ["nombre"].str.lower().sort_values(ascending=True).index]

mineduc_univ['nombre'] = mineduc_univ['nombre'].str.lower()

mineduc_univ['nombre']
#['nombre'].sort_values(axis=0, ascending=False)

48                        pontificia universidad católica de chile
51                   pontificia universidad católica de valparaíso
7                      universidad academia de humanismo cristiano
14                                       universidad adolfo ibañez
20                                 universidad adventista de chile
31                                     universidad alberto hurtado
12                                        universidad andrés bello
43                                         universidad arturo prat
52                                    universidad austral de chile
18                                   universidad autónoma de chile
28                                  universidad bernardo o'higgins
4                                          universidad bolivariana
23              universidad católica cardenal raúl silva henríquez
55                 universidad católica de la santísima concepción
56                                  universidad católica de te

In [21]:
#mineduc_univ.columns = mineduc_univ.columns.str.upper()

mineduc_univ.rename(columns={'tipo':'tipo_nuevo'})

,tipo_nuevo,n_registro,nombre,tipo_reconocimiento,documento_reconocimiento,n_documento_reconocimiento,fecha_reconocimiento,regimen_funcionamiento,año_reconocimiento
48,B,1,pontificia universidad católica de chile,Fundación,Decreto,S/N,1888-06-21,Autónoma,1888
51,B,4,pontificia universidad católica de valparaíso,Creación,Decreto,s/n,1928-03-15,Autónoma,1928
7,C,14,universidad academia de humanismo cristiano,Autorización de Funcionamiento,Certificado,783,1988-08-26,Autónoma,1988
14,C,26,universidad adolfo ibañez,Autorización de Funcionamiento,Certificado,38,1989-01-13,Autónoma,1989
20,C,42,universidad adventista de chile,Autorización de Funcionamiento,Certificado,119,1990-06-02,Autónoma,1990
31,C,74,universidad alberto hurtado,Reconocimiento Oficial,Dex.,1001,1997-03-10,Autónoma,1997
12,C,23,universidad andrés bello,Autorización de Funcionamiento,Oficio,1077,1988-12-29,Autónoma,1988
43,A,12,universidad arturo prat,Creación,Ley,"18,368",1984-12-14,Autónoma,1984
52,B,5,universidad austral de chile,Fundación,Decreto,3757,1954-07-09,Autónoma,1954
18,C,35,universidad autónoma de chile,Autorización de Funcionamiento,Certificado,102,1990-01-31,Autónoma,1990


## Cargamos los archivos Fondecyt regular

Estos archivos vienen en format csv. Los nombres que aparecen en la columna nombre vienen en mayúsculas y no son completamente oficiales.

El objetivo es hacer un match entre la columna nombre de estos archivos y la columna nombre de Mineduc para así quedarnos con el nombre oficial reconocido por Mineduc

In [16]:
fondecyt_2010=pd.read_csv('data/fondecyt/fondecyt regular 2010.csv', header=0, sep=',')
fondecyt_2011=pd.read_csv('data/fondecyt/fondecyt regular 2011.csv', header=0, sep=',')
fondecyt_2012=pd.read_csv('data/fondecyt/fondecyt regular 2012.csv', header=0, sep=',')
fondecyt_2013=pd.read_csv('data/fondecyt/fondecyt regular 2013.csv', header=0, sep=',')
fondecyt_2014=pd.read_csv('data/fondecyt/fondecyt regular 2014.csv', header=0, sep=',')
fondecyt_2015=pd.read_csv('data/fondecyt/fondecyt regular 2015.csv', header=0, sep=',')
fondecyt_2016=pd.read_csv('data/fondecyt/fondecyt regular 2016.csv', header=0, sep=',')

In [ ]:
fondecyt_2010.head()

Reemplazamos algunas abreviaciones por el nombre completo

In [ ]:
fondecyt_regular=pd.concat([fondecyt_2010,fondecyt_2011,fondecyt_2012,fondecyt_2013,fondecyt_2014,fondecyt_2015,fondecyt_2016]).reset_index(drop=True)
for i in range(len(fondecyt_regular.index)):
    name=(fondecyt_regular.loc[i,'nombre']).upper()
    name=name.replace("UNIV.","UNIVERSIDAD").replace("PONT.","PONTIFICIA").replace("CS.","CIENCIAS").replace("TEC.","TECNOLOGIA")
    name=name.replace("INTERNACIONAL SEK","SEK").replace("UNIVERSIDAD CIENCIAS DE LA INFORMATICA","UNIVERSIDAD UCINF")
    fondecyt_regular.loc[i,'nombre']=name.capitalize()

fondecyt_regular.head()

Hacemos el match usando el paquete fuzzyword

In [ ]:
for i in range(len(fondecyt_regular.index)):
    if fondecyt_regular.loc[i,'nombre'] != 'Otras universidades':
        d = mineduc.apply(lambda x: fuzz.ratio(x['nombre'].upper(), fondecyt_regular.loc[i, 'nombre'].upper()), axis=1)
        if d.max()<90:
            print("WARNING - Fuzz ratio lower than 90")
            print('Fondecyt name: ', fondecyt_regular.loc[i,'nombre'])
            print('Mineduc name: ', mineduc.loc[d.idxmax(), 'nombre'])
        fondecyt_regular.loc[i,'nombre']=mineduc.loc[d.idxmax(), 'nombre']

fondecyt_regular.head()

## Realizamos algunas operaciones de agregación

Agrupemos por nombre de Universidad

In [ ]:
fondecyt_univ = fondecyt_regular.groupby(['nombre']).sum().sort_values('n_aprobados', ascending=False)
fondecyt_univ.head()

In [ ]:
fondecyt_univ.to_csv('results/Fondecyt_agrupado_por_universidades', index=False)

Agrupemos por nombre de Universidad y año de adjudicación

In [ ]:
fondecyt_univ_year=fondecyt_regular.groupby(['nombre','año']).sum()  #.sort_values('n_aprobados',ascending=False)[['n_concursados','n_aprobados']]
fondecyt_univ_year['tasa_aprobacion']=np.round(fondecyt_univ_year['n_aprobados']/fondecyt_university['n_concursados']*100.,decimals=1)
fondecyt_univ_year.head()

Eliminamos las Universidades con nombre "Otras Universidades"

In [ ]:
fondecyt_regular = fondecyt_regular[fondecyt_regular['nombre']!= "Otras universidades"]
fondecyt_regular.head()

In [ ]:
fondecyt_univ_year=fondecyt_regular.groupby(['nombre','año']).sum()  #.sort_values('n_aprobados',ascending=False)[['n_concursados','n_aprobados']]
fondecyt_univ_year['tasa_aprobacion']=np.round(fondecyt_univ_year['n_aprobados']/fondecyt_university['n_concursados']*100.,decimals=1)
fondecyt_univ_year.head(10)

Algunas veces es conveniente tener los años como campos de la tabla

In [ ]:
table = pd.pivot_table(fondecyt_regular,values=['n_concursados','n_aprobados'],index=['nombre'],columns=['año'],aggfunc=np.sum,margins=True)
#table = table.round({'tasa_aprobacion':1})
table.sort_values(by=('n_aprobados','All'), ascending=False,inplace=True)
table.reset_index( inplace=True)
table
